Nesta atividade construiremos modelos preditivos de regressão em scikit learn para a predição dos votos de deputados federais considerando as últimas eleições. As atividades esperadas para essa etapa são descritas a seguir:

1. Baixe os dados aqui.

Vamos explorar dados sobre as votações que candidatos à Câmara Federal de Deputados receberam nos anos de 2006 e 2010. Esses dados foram extraídos do TSE (Links para um site externo)Links para um site externo, pré-processados e contemplam informações sobre aproximadamente 7.300 candidatos.

2. Considere o pipeline mostrado nesse [link](https://www.kaggle.com/apapiu/regularized-linear-models) para um site externo para construir seus modelos de regressão. Isso implica, dentre outras coisas:

    1. Analisar as distribuições das variáveis para ver se estão enviesadas e precisam de correção; tratamento de valores ausentes, variáveis categóricas e normalização, quando for o caso.
    2. Construir modelos de regressão com (ridge e lasso) e sem regularização.
    3. Considerar também modelos de regressão não paramétrica como K-NN.
    4. Considerar outros modelos ainda não vistos em sala de sua escolha (e.g. SVR, Regression Trees e Random Florests).
    5. Tunar os hiperâmetros para cada caso e retornar os rmses de validação cruzada para todos os modelos avaliados.
    6. Plotar os resíduos versus predições e analisar se esses plots representam bons indícios da adequabilidade dos modelos a esse problema.
    
3. Alguns dias antes da entrega final serão liberados os dados de teste referentes à 2014 para validação final dos seus melhores modelos.
    1. Dica: Uma coisa que você pode fazer é usar os dados de 2006 como treino e os de 2010 como validação. Uma vez encontrados os melhores modelos para 2010 junte 2006+2010, retreine, e aplique o modelo aos dados de 2014 que serão liberados.
    
4. Responder:
    1. Dentre os modelos avaliados, qual foi o que deu o melhor resultado nos dados de 2014 em termos de RMSE? Justifique bem sua resposta.

A entrega deve ser um notebook Jupyter com código python e texto explicativo quando necessário. Crie um repositório na sua conta do github e envie o link do html do notebook.


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib

import matplotlib.pyplot as plt
from scipy.stats import skew

from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

## Análise dos dados 

### Leitura dos dados

In [3]:
train_data = pd.read_csv("dados/eleicoes_2006_a_2010.csv")
#test_data = pd.read_csv("dados/eleicoes_2006_a_2010.csv") #TODO change

# load the dataset as a pandas data frame
df_train_data = pd.DataFrame(train_data)

df_data = df_train_data #TODO remove this line
#TODO and uncommet the lines below
#df_test_data = pd.DataFrame(test_data)

#frames = [df_train_data, df_test_data]
#df_data = pd.concat(frames)

In [4]:
df_data.head()

,ano,sequencial_candidato,nome,uf,partido,quantidade_doacoes,quantidade_doadores,total_receita,media_receita,recursos_de_outros_candidatos/comites,...,quantidade_despesas,quantidade_fornecedores,total_despesa,media_despesa,cargo,sexo,grau,estado_civil,ocupacao,votos
0,2006,10001,JOSÉ LUIZ NOGUEIRA DE SOUSA,AP,PT,6,6,16600.00,2766.666667,NaN,...,14,14,16583.60,1184.542857,DEPUTADO FEDERAL,MASCULINO,ENSINO MÉDIO COMPLETO,CASADO(A),VEREADOR,8579
1,2006,10002,LOIVA DE OLIVEIRA,RO,PT,13,13,22826.00,1755.846154,6625.0,...,24,23,20325.99,883.738696,DEPUTADO FEDERAL,FEMININO,SUPERIOR COMPLETO,SOLTEIRO(A),SERVIDOR PÚBLICO ESTADUAL,2757
2,2006,10002,MARIA DALVA DE SOUZA FIGUEIREDO,AP,PT,17,16,148120.80,9257.550000,2250.0,...,123,108,146011.70,1351.960185,DEPUTADO FEDERAL,FEMININO,SUPERIOR COMPLETO,VIÚVO(A),PEDAGOGO,17428
3,2006,10002,ROMALDO MILANI,MS,PRONA,6,6,3001.12,500.186667,NaN,...,8,8,3001.12,375.140000,DEPUTADO FEDERAL,MASCULINO,ENSINO MÉDIO INCOMPLETO,CASADO(A),MILITAR REFORMADO,1193
4,2006,10003,ANSELMO DE JESUS ABREU,RO,PT,48,48,NaN,NaN,NaN,...,133,120,116416.64,970.138667,DEPUTADO FEDERAL,MASCULINO,ENSINO FUNDAMENTAL COMPLETO,CASADO(A),DEPUTADO,29401


### Pré-processamento dos dados

- Transformação das features numéricas enviesadas usando log(feature + 1), a fim de normalizar a distribuição dos valores;
- Tratamento das features numéricas com valores ausentes (NaN's);
- Transformação das variáveis categóricas.

#### Enviesamento de features numéricas

Através dos histogramas abaixo vemos que as variáveis analisadas precisam de normalização, porque a distribuição dos seus valores não está igual.

In [5]:
# function to plot a histogram of each variable
def plot_histogram(feature):
    # log (feature + 1) because of features with values 0
    feature = pd.DataFrame({"1. " + feature: df_data[feature], "2. log(" + feature + ")": np.log1p(df_data[feature])})
    feature.hist() # TODO check how to increase the number of bins

# function to transform skewed variables using log(feature + 1)
def transform_skewed_var(feature):
    df_data[feature] = np.log1p(df_data[feature])

In [6]:
columns = "quantidade_doacoes quantidade_doadores total_receita media_receita recursos_de_outros_candidatos/comites recursos_de_pessoas_fisicas recursos_de_pessoas_juridicas recursos_proprios quantidade_despesas quantidade_fornecedores total_despesa media_despesa".split()

for feature in columns:
    plot_histogram(feature)
    transform_skewed_var(feature)    

#### Features com valores numéricos ausentes (NaN) 

Abaixo veremos quais as variáveis com valores ausentes (NaN).

In [7]:
df_data.isnull().any()

ano                                      False
sequencial_candidato                     False
nome                                     False
uf                                       False
partido                                  False
quantidade_doacoes                       False
quantidade_doadores                      False
total_receita                             True
media_receita                             True
recursos_de_outros_candidatos/comites     True
recursos_de_pessoas_fisicas               True
recursos_de_pessoas_juridicas             True
recursos_proprios                         True
quantidade_despesas                      False
quantidade_fornecedores                  False
total_despesa                            False
media_despesa                            False
cargo                                    False
sexo                                     False
grau                                     False
estado_civil                             False
ocupacao     

In [8]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7330 entries, 0 to 7329
Data columns (total 23 columns):
ano                                      7330 non-null int64
sequencial_candidato                     7330 non-null int64
nome                                     7330 non-null object
uf                                       7330 non-null object
partido                                  7330 non-null object
quantidade_doacoes                       7330 non-null float64
quantidade_doadores                      7330 non-null float64
total_receita                            7169 non-null float64
media_receita                            7169 non-null float64
recursos_de_outros_candidatos/comites    4940 non-null float64
recursos_de_pessoas_fisicas              4647 non-null float64
recursos_de_pessoas_juridicas            3354 non-null float64
recursos_proprios                        4942 non-null float64
quantidade_despesas                      7330 non-null float64
quantidade_fornece

Das 6 variáveis com valores ausentes,  **recursos_de_outros_candidatos/comites**, **recursos_de_pessoas_fisicas**, **recursos_de_pessoas_juridicas** e **recursos_proprios** são as que mais possuem valores ausentes.

O gráfico mais abaixo mostra a correlação entre todas as variáveis numéricas. Nele, quanto mais amarelo maior a correlação entre a variável do eixo horizontal e a variável do eixo vertical.

In [9]:
#analysis of correlation between variables

corr = df_data.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)
cmap='viridis'

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=0.4, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

Vemos acima que a variável que mais tem correlação com as outras é **total_receita**.

Apesar dela também ter valores ausentes, optou-se por remover dos dados apenas as outras 4 variáveis que tem mais valores ausentes, mas tem alta correlação com **total_receita**. Nesta última e em **media_receita**, os valores ausentes serão substituídos pela mediana de todos os valores de cada coluna.

In [10]:
print(df_data.shape)

#removing collumns
df_data = df_data.drop(["nome", "recursos_de_outros_candidatos/comites",
           "recursos_de_pessoas_fisicas","recursos_de_pessoas_juridicas",
           "recursos_proprios"], 1)

#filling NA's of total_receita and media_receita with the median of the column:
df_data = df_data.fillna(df_data.median())

print(df_data.shape)
df_data.isnull().any()

(7330, 23)
(7330, 18)


ano                        False
sequencial_candidato       False
uf                         False
partido                    False
quantidade_doacoes         False
quantidade_doadores        False
total_receita              False
media_receita              False
quantidade_despesas        False
quantidade_fornecedores    False
total_despesa              False
media_despesa              False
cargo                      False
sexo                       False
grau                       False
estado_civil               False
ocupacao                   False
votos                      False
dtype: bool

#### Tranformação das variáveis categóricas

As variáveis categóricas presentes nos dados, como **ocupação** e **estado_civil**, serão transformadas em variáveis numéricas, a fim de permitir a sua utilização nos modelos de regressão logo a seguir.

In [11]:
df_data = pd.get_dummies(df_data)

df_data.head(10)

,ano,sequencial_candidato,quantidade_doacoes,quantidade_doadores,total_receita,media_receita,quantidade_despesas,quantidade_fornecedores,total_despesa,media_despesa,...,"ocupacao_TÉCNICO DE OBRAS CIVIS, ESTRADAS, SANEAMENTO E ASSEMELHADOS",ocupacao_TÉCNICO EM AGRONOMIA E AGRIMENSURA,ocupacao_TÉCNICO EM EDIFICAÇÕES,ocupacao_TÉCNICO EM INFORMÁTICA,ocupacao_VENDEDOR DE COMÉRCIO VAREJISTA E ATACADISTA,"ocupacao_VENDEDOR PRACISTA, REPRESENTANTE, CAIXEIRO-VIAJANTE E ASSEMELHADOS",ocupacao_VEREADOR,ocupacao_VETERINÁRIO,ocupacao_VIGILANTE,ocupacao_ZOOTECNISTA
0,2006,10001,1.945910,1.945910,9.717218,7.925760,2.708050,2.708050,9.716230,7.077956,...,0,0,0,0,0,0,1,0,0,0
1,2006,10002,2.639057,2.639057,10.035699,7.471276,3.218876,3.178054,9.919705,6.785292,...,0,0,0,0,0,0,0,0,0,0
2,2006,10002,2.890372,2.833213,11.905790,9.133303,4.820282,4.691348,11.891449,7.210050,...,0,0,0,0,0,0,0,0,0,0
3,2006,10002,1.945910,1.945910,8.007074,6.216979,2.197225,2.197225,8.007074,5.929961,...,0,0,0,0,0,0,0,0,0,0
4,2006,10003,3.891820,3.891820,9.400695,7.340539,4.897840,4.795791,11.664939,6.878469,...,0,0,0,0,0,0,0,0,0,0
5,2006,10003,1.945910,1.945910,7.479983,5.691041,2.302585,2.302585,7.479983,5.287262,...,0,0,0,0,0,0,0,0,0,0
6,2006,10003,2.708050,2.079442,6.548219,4.610868,2.890372,2.397895,6.548090,4.258318,...,0,0,0,0,0,0,0,0,0,0
7,2006,10003,1.098612,1.098612,11.156265,10.463132,5.808142,5.631212,11.097682,5.474248,...,0,0,0,0,0,0,0,0,0,0
8,2006,10004,1.098612,1.098612,8.710455,8.017472,2.397895,2.397895,8.709699,6.408598,...,0,0,0,0,0,0,0,0,0,0
9,2006,10004,4.234107,4.189655,11.569440,7.395657,4.234107,4.219508,11.615655,7.411558,...,0,0,0,0,0,0,0,0,0,0


### Divisão dos dados em treinamento, validação e teste

Abaixo, o conjunto de dados utilizado será particionado em treino, para servir para a criação dos modelos de regressão, em validação, que será usado para avaliar a qualidade do modelo e em teste, que servirá para avaliar as predições do modelo para dados ainda não vistos por ele.

In [12]:
data_train = df_data.loc[df_data["ano"] == 2006]
y_train = data_train.votos
X_train = data_train.drop("votos", 1)
print(X_train.shape)
print(y_train.shape)

data_validation = df_data.loc[df_data["ano"] == 2010]
y_validation = data_validation.votos
X_validation = data_validation.drop("votos", 1)
print(X_validation.shape)
print(y_validation.shape)

#TODO uncomment for test with new data
#data_test = df_data.loc[df_data["ano"] == 2014]
#y_test = data_test.votos
#X_test = data_test.drop("votos", 1)
#print(X_test.shape)
#print(y_test.shape)

(3388, 258)
(3388,)
(3942, 258)
(3942,)


## Implementação dos Modelos de regressão

### 1. Modelos de regressão sem regularização

In [13]:
# function to calculate the RMSE of the model with cross validation
def rmse_cv(model):
    rmse = np.sqrt(-cross_val_score(model, X_train, y_train, scoring = "neg_mean_squared_error", cv = 5))
    return(rmse)

In [14]:
def print_coefficients(model):
    w = list(model.coef_)
    # Parameters must be in reverse order because the poly function 
    w.reverse()
    print (np.poly1d(w) + model.intercept_) 

In [15]:
def plot_rmse_param(series, param_name):
    series.plot(title = "Erro de validação vs " + param_name)
    plt.xlabel(param_name)
    plt.ylabel("rmse")

In [16]:
def best_rmse_param(series):
    best_rmse = series.min()
    best_param = series.idxmin() 
    
    return(best_rmse, best_param)

In [17]:
def train_model(model, print_coef):
    model.fit(X_train, y_train)
    if (print_coef):
        print_coefficients(model)
        
    print("Score: ", model.score(X_validation, y_validation)) #TODO change to X_test, y_test

#### 1.1 Implementando o modelo de regressão Ridge

In [18]:
model_ridge = Ridge()
train_model(model_ridge, True)

       257             256        255        254        253        252
-7182 x   + 1.227e+04 x   + 1515 x   - 6077 x   - 4624 x   - 1420 x  
         251             250        249        248             247
 - 3426 x   + 9.758e+04 x   - 2609 x   - 5151 x   - 1.561e+04 x  
         245        244        243        242        241
 - 4161 x   - 4458 x   + 3918 x   + 1909 x   - 2080 x  
              239         238             237             232
 + 2.616e+04 x   - 881.5 x   - 1.788e+04 x   - 1.037e+04 x  
              231        229        228        227        226
 + 1.526e+04 x   - 8886 x   - 3634 x   + 3960 x   - 2816 x  
              225        224        223        222        221
 - 1.775e+04 x   + 3050 x   - 6233 x   - 5659 x   - 1593 x  
         220        219        217             215         213
 - 1349 x   - 8513 x   + 2853 x   + 1.331e+04 x   - 845.3 x  
          212        211        209        208        207
 - 851.9 x   - 4252 x   - 9039 x   - 7267 x   - 5943 x  
    

O hiperparâmetro do Rigde é o alpha - um parâmetro de regularização que mede quão flexível é o modelo. Quanto maior a regularização, menos propenso será o nosso modelo de overfitting. No entanto, também perderá a flexibilidade e poderá não capturar todo o padrão dos dados.

Testaremos alguns valores para alpha e usaremos o melhor para treinar o modelo.

In [19]:
alphas = [0.001, 0.1, 0.5, 1, 5, 20, 25]
cv_ridge_rmse = [rmse_cv(Ridge(alpha = alpha, max_iter = None, solver = "sag")).mean() 
            for alpha in alphas]

series_ridge = pd.Series(cv_ridge_rmse, index = alphas)
plot_rmse_param(series_ridge, "alpha")

Observe a curva em forma de U acima. Quando o alpha é muito grande, a regularização é muito forte e o modelo não consegue capturar todas as complexidades dos dados - o erro volta a aumentar. Se, no entanto, deixarmos o modelo ser muito flexível (alpha pequeno), o modelo tende ao overfitting. Um valor de alpha = 30 parece o melhor, com base no gráfico acima.

In [20]:
best_rmse_ridge, best_param_ridge = best_rmse_param(series_ridge)
print(best_rmse_ridge, best_param_ridge)

36127.5141782 0.001


In [21]:
model_ridge = Ridge(alpha = best_param_ridge, fit_intercept = True, max_iter = None, solver = "sag")
train_model(model_ridge, True)

        257          256         255        254         253         252
-1.039 x   - 0.1242 x   + 3.867 x   - 40.7 x   - 1.671 x   - 1.769 x  
          251         250          249          248          247
 + 0.666 x   + 8.334 x   - 0.6453 x   - 0.4438 x   - 0.6333 x  
           245         244          243          242         241
 - 0.8642 x   - 2.317 x   - 0.5728 x   - 0.3051 x   - 2.006 x  
          239          238         237          232         231
 + 10.68 x   - 0.1568 x   - 1.101 x   - 0.6544 x   + 3.228 x  
           229         228         227            226          225
 - 0.9206 x   - 0.538 x   - 0.969 x   - 0.007606 x   - 0.9547 x  
           224         223         222        221         220
 - 0.3287 x   - 3.999 x   - 12.04 x   - 27.8 x   - 35.99 x  
          219           217         215         213         212
 - 15.94 x   - 0.03676 x   + 1.363 x   - 1.099 x   - 5.015 x  
          211        209         208         207           206
 - 7.375 x   - 1.56 x   - 

#### 1.2 Implementando o modelo de regressão Lasso

In [22]:
model_lasso = Lasso(max_iter = 10000)
train_model(model_lasso, True)

       257             256        255        254        253        251
-8653 x   + 1.516e+04 x   + 2801 x   - 4340 x   - 3439 x   - 2962 x  
              250         249        248             247        245
 + 1.942e+05 x   - 15.25 x   - 4479 x   - 2.726e+04 x   - 3046 x  
         244        243        242         241             239
 - 3252 x   + 5411 x   + 2385 x   - 239.2 x   + 3.173e+04 x  
              237             232            231            229
 - 3.097e+04 x   - 1.289e+04 x   + 1.97e+04 x   - 1.21e+04 x  
         228        227        226             225        224
 - 2480 x   + 6057 x   - 1117 x   - 3.167e+04 x   + 5211 x  
         223        222         220        219        217
 - 5147 x   - 4327 x   + 174.6 x   - 7104 x   + 3606 x  
              215         212        211             209        208
 + 2.431e+04 x   + 705.6 x   - 2676 x   - 1.037e+04 x   - 6248 x  
         207             206         204        203         202
 - 5033 x   + 2.509e+04 x   + 731.4

Vamos agora usar o Lasso CV para descobrir o melhor alpha para nós.

In [23]:
alphas = [0.1, 0.5, 1, 5, 10]
cv_lasso_rmse = [rmse_cv(Lasso(alpha = alpha, max_iter = 10000, tol=0.01)).mean() 
            for alpha in alphas]

series_lasso = pd.Series(cv_lasso_rmse, index = alphas)
plot_rmse_param(series_lasso, "alpha")

In [24]:
best_rmse_lasso, best_param_lasso = best_rmse_param(series_lasso)
print(best_rmse_lasso, best_param_lasso)

33314.5058278 10.0


In [25]:
model_lasso = Lasso(alpha = best_param_lasso, fit_intercept = True, max_iter = 10000, tol=0.01)
train_model(model_lasso, True)

           256        254         253             250             239
1.035e+04 x   - 3614 x   - 343.9 x   + 1.647e+05 x   + 2.701e+04 x  
              231         223        222         221         220
 + 1.337e+04 x   - 21.78 x   - 2885 x   + 272.4 x   + 476.2 x  
         219         211        208        203        199        196
 - 5692 x   - 588.7 x   - 2704 x   - 1772 x   - 7053 x   + 3787 x  
         195        191        190         188        183
 + 1851 x   - 1459 x   - 1427 x   - 157.7 x   - 1656 x  
              181        180       177        167         165
 + 2.392e+04 x   + 2701 x   + 394 x   + 1329 x   - 363.9 x  
         161        159         152        150         148        144
 - 8697 x   - 1817 x   + 264.2 x   - 9702 x   + 577.3 x   - 3067 x  
         139         136         134        131             130
 + 9005 x   - 508.5 x   - 815.4 x   - 7951 x   - 1.037e+04 x  
         127             125         121        119        107
 + 5480 x   + 3.159e+04 x   

### 2. Modelos de regressão não parámetrica

#### 2.1 Implementando o modelo de regressão K-NN

In [26]:
model_knn = KNeighborsClassifier()
train_model(model_knn, False)

Score:  0.00025367833587


A quantidade de vizinhos (k) é o parâmetro mais importante do algoritmo K-NN e define a quantidade de valores vizinhos de cada valor de entrada que o algoritmo deve considerar para estimar a saída. Por isso, usaremos cross-validation para encontrar o melhor valor deste parâmetro.

In [27]:
n_neighbors = [3, 5, 10, 25, 50]
cv_knn_rmse = [rmse_cv(KNeighborsClassifier(n_neighbors = n)).mean() 
            for n in n_neighbors]

series_knn = pd.Series(cv_knn_rmse, index = n_neighbors)
plot_rmse_param(series_knn, "n_neighbors")

C:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be l

In [28]:
best_rmse_knn, best_param_knn = best_rmse_param(series_knn)
print(best_rmse_knn, best_param_knn)

12589.5150163 5


In [29]:
model_knn = KNeighborsClassifier(n_neighbors = best_param_knn)
train_model(model_knn, False)

Score:  0.00025367833587


### 3. Outros modelos de regressão

#### 3.1 Implementando o modelo SVR (Suport Vector Regression)

In [30]:
model_svr = SVR()
train_model(model_svr, False)

Score:  -0.142025836426


O C é um parâmetro de penalidade importante no SVR, porque se for muito grande poderemos ter overffiting ou underffiting, caso contrário. Por isso, tentaremos encontrar o seu melhor valor.

In [31]:
cs = [1, 5, 10, 25, 50]
cv_svr_rmse = [rmse_cv(SVR(C = c)).mean() 
            for c in cs]

series_svr = pd.Series(cv_svr_rmse, index = cs)
plot_rmse_param(series_svr, "C")

In [32]:
best_rmse_svr, best_param_svr = best_rmse_param(series_svr)
print(best_rmse_svr, best_param_svr)

48853.3668547 1


In [33]:
model_svr = SVR(C = best_param_svr)
train_model(model_svr, False)

Score:  -0.142025836426


#### 3.2 Implementando o modelo de regressão Random Forest

In [34]:
model_rf = RandomForestClassifier()
train_model(model_rf, False)

Score:  0.00025367833587


Um dos parâmetros mais importantes do Random Forest é a quantidade de estimadores, que representam a quantidade de árvores usadas no modelo. Em geral, quanto maior melhor o desempenho do modelo, porém mais memória é necessário.

Abaixo, encontramos o melhor valor dado um intervalo de 1 à 20.

In [35]:
n_estimators = [1, 5, 10, 15, 20]
cv_rf_rmse = [rmse_cv(RandomForestClassifier(n_estimators = n)).mean() 
            for n in n_estimators]

series = pd.Series(cv_rf_rmse, index = n_estimators)
plot_rmse_param(series, "n_estimators")

C:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be l

In [36]:
best_rmse_rf, best_param_rf = best_rmse_param(series)
print(best_rmse_rf, best_param_rf)

12081.5076358 10


In [37]:
model_rf = RandomForestClassifier(n_estimators = best_param_rf)
train_model(model_rf, False)

Score:  0.0


## Análise da predição dos modelos

In [38]:
# function to plot the residual vs predictions values of the model
def plot_res_vs_pred(model):
    y_pred = model.predict(X_validation)
    res = y_validation - y_pred
    #y_pred = model.predict(X_test)
    #res = Y_test - y_pred
    plt.plot(y_pred, res, 'k.', color='blue')
    plt.axhline(y=0., color='r', linestyle='-')
    plt.xlabel("predictions")
    plt.ylabel("residuals")

### Ridge

In [39]:
plot_res_vs_pred(model_ridge)

### Lasso

In [40]:
plot_res_vs_pred(model_lasso)

### K-NN

In [41]:
plot_res_vs_pred(model_knn)

### SVR

In [42]:
plot_res_vs_pred(model_svr)

### Random Forest

In [43]:
plot_res_vs_pred(model_rf)

## Conclusões